# Bölüm 1 Hazırlık

# Kütüphaneleri İndirme

In [ ]:
# Import libs
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Veriyi Yükleme

In [ ]:
# Import dataset
dataset = pd.read_csv("C:\\Users\\toshiba\\SkyDrive\\veribilimi.co\\Datasets\\Udemy_A_Z_Deep_Learning\\Churn_Modelling.csv")

In [ ]:
# veri setine bakış
dataset.head()

# Exited hedef değişken

In [ ]:
len(dataset.columns)

In [ ]:
# Bağımlı ve bağımsız değişkenleri ayırma
# RowNumber, CustomerId, Surname hedef değişkeni tahmin etme bakımından işe yaramaz. Bu yüzden 0,1,2 indeksli nitelikleri
# Nitelikler matrisine dahil etmiyoruz. Yukarıda kaç tane nitelik olduğunu yazdırmıştık. Şimdi 0,1,2 hari ise 3'ten başlayacak. 
# Python'da indeks değeri 0'dan başladığına ve 14 nitalik olduğuna göre son niteliğin indeks değeri 13 olacaktır. iloc ile sütun
#seçerken ilk değer dahil ikinci değer hariçtir. bu sebeple 3'ü dahil 13'ü hariç tutacağız. Çünkü son indeks değeri Exited yani
# hedef değişkene geliyor. İlk : tüm satırlar "," sonrası sütunlardan seçilecek filtre. 3:13 demek 3 dahil 13 hariç indeksli
# sütunları seç demek

X = dataset.iloc[:,3:13].values

# Hedef niteliğimiz 13. indekste idi. O sebeple : ile tüm satırları "," den sonra 13 ile hedef niteliğin indeksini seçiyoruz.
y = dataset.iloc[:,13].values

In [ ]:
X

In [ ]:
y

In [ ]:
type(X)

In [ ]:
type(y)


# Veri setini eğitim ve test olarak bölmeden önce mutlaka kategorik nitelikleri halletmek gerekir.

In [ ]:
# Hedef nitelik 0 ve 1 olmak üzere zaten nümerik değer olduğu için onu LabelEncoder ile kodlamaya gerek yok.
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# X nitelikler matrisini incelediğimizde 1'inci indekste Geography 2'nci indekste Gender'ın kategorik olduğunu görüyoruz.
# Önce Geagraphy için LabelEncoder yaratalım
labelEncoder_Geo = LabelEncoder()
X[:, 1] = labelEncoder_Geo.fit_transform(X[:, 1])

# Şimdi Gender için
labelEncoder_Gender = LabelEncoder()
X[:,2] = labelEncoder_Gender.fit_transform(X[:,2])

# Gender binary olduğu için onu oneHot yapmaya gerek yok. Ancak Geography üç değerli olduğu için onu oneHot (Gölge değişken)
# yapmalıyız. Bunun için OneHotEncoder sınıfını kullanacağız.
# Nesneyi oluşturalım ve dönüştüreceğimiz sütunların indeksinden oluşan listeyi parametre verelim. Burada sadece [1] çünkü
# tek bir tane 
oneHotEncoder = OneHotEncoder(categorical_features=[1])

# nesnemizin fit_transform() metodunu kullanarak Geography niteliğini gölge değişkenlere çeviriyoruz ve X'e ekliyoruz.
X = oneHotEncoder.fit_transform(X).toarray()

In [ ]:
X.shape

In [ ]:
# Gölge Değişken Tuzağından kurtulmak için Yeni oluşturulan gölge değişkenlerden birini çıkaralım
X = X[:,1:]

In [ ]:
X.shape

# Eğitim ve Test Seti olarak Ayırmak

In [ ]:
from sklearn.model_selection import train_test_split

#Sıralamaya dikkat. Aynı sonuçlar için random_state değeri aynı olmalıdır.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0)

# Feature Scaling

In [ ]:
# Nitelikler matrisini (X) StandardScaler
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Evaluate, Improve and Tune ANN

In [ ]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense

In [ ]:
def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(kernel_initializer = 'uniform', input_dim = 11, units = 6,  activation = 'relu'))
    classifier.add(Dense(kernel_initializer = 'uniform', units = 4,  activation = 'relu'))
    classifier.add(Dense(kernel_initializer = 'uniform', units = 1,  activation = 'sigmoid'))
    classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return classifier

# Fonksiyondan sonra fonksiyon dışında da bir nesne yaratmalıyız. Çünkü o fonksiyon içinde kaldı.
classifier = KerasClassifier(build_fn=build_classifier, batch_size=10, epochs=100)

In [ ]:
# şimdi cv sonuçlarını alacak bir değişken yaratalım
# estimator az önce KerasClassifier sınıfından yarattığımız classifier, X_train ve y_train belli zaten
# cv=10 crossvalidation rakamı, n_jobs kaç tane işlemci çekirdeği kullansın -1 müsait olan hepsini kullan demek

accuracies = cross_val_score(estimator=classifier, X = X_train, y = y_train, cv=10, n_jobs=-1)

In [ ]:
mean = accuracies.mean()
variance = accuracies.std()